In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Luminesce Syntax

Attributes
----------

"""

toggle_code("Toggle Docstring")


*Setting up the environment for the tutorial*

This section of the notebook creates a portfolio, some equities, and a file in drive to run luminesce queries against.

In [2]:
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
import lusid as lu
import pandas as pd
import lusid_drive
import lusid_drive.rest
import lusid.rest
from lusid_drive.rest import ApiException
from pprint import pprint
import json


secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = ApiClientFactory(
    token = RefreshingToken(), 
    api_secrets_filename = secrets_path,
    app_name = "LusidJupyterNotebook"
)

# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.11152.0,0.5.3222,"{'relation': 'RequestLogs', 'href': 'http://fb..."


In [3]:
# here's some sample equities we've prepared earlier
uk_equity_instruments = pd.read_csv('_data/uk_equity_instruments.csv')
us_equity_instruments = pd.read_csv('_data/us_equity_instruments.csv')

In [4]:
# don't show exception if error is due to upsert
def exception_guard(e, code):
    return e.status and e.status != '400 Bad Request' and e.body and json.loads(e.body)["code"] == code

scope = 'FBNUniversityModule-T2-2' # str | The scope of the portfolio.

In [5]:
# create a request to upsert a bunch of instruments.
def generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier):
    request_id = name + isin
    request ={"name":name,
                     "identifiers":{ "Figi":{"value":figi},
                                     "Isin":{"value":isin},
                                     other_identifier_name:{"value":other_identifier}
                                   },

                     "definition":{"instrumentType":"Equity", "domCcy":ccy}}
    return request_id, request

In [6]:
# Create an instance of the API class
api_instance = api_factory.build(lu.InstrumentsApi)

# We're going to load some american and UK equities into LUSID

other_identifier_name = 'Sedol'
ccy = 'GBP'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in uk_equity_instruments.itertuples())
uk_equity_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}
other_identifier_name = 'Cusip'
ccy = 'USD'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in us_equity_instruments.itertuples())
us_equity_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}

request_body_dict = us_equity_body_dict | uk_equity_body_dict

try:
    # UpsertInstruments: Upsert instruments
    api_response = api_instance.upsert_instruments(request_body_dict, scope=scope)
#     pprint(api_response)
except lusid.rest.ApiException as e:
    print("Exception when calling InstrumentsApi->upsert_instruments: %s\n" % e)


In [7]:

# let's chuck a file containing a randomly selected subset of 25 instruments with their sectors into drive for some examples:

subset_file = open('_data/instrument_subset.csv').read()

drive_config = lusid_drive.Configuration(
    host = f'{api_instance.api_client.configuration.host[:-4]}/drive'
)
drive_config.access_token = api_factory.api_client.configuration.access_token

# Enter a context with an instance of the API client
with lusid_drive.ApiClient(drive_config) as api_client:
    # Create an instance of the API class
    api_instance = lusid_drive.FoldersApi(api_client)
    create_folder = {"path":"/","name":"finbourne university"} # CreateFolder | A CreateFolder object that defines the name and path of the new folder

    try:
        # [EARLY ACCESS] CreateFolder: Create a new folder in LUSID Drive
        api_response = api_instance.create_folder(create_folder)
        pprint(api_response)
    except ApiException as e:
        if not exception_guard(e, 664):
            print("Exception when calling FilesApi->create_file: %s\n" % e)
        
        
    api_instance = lusid_drive.FilesApi(api_client)
    x_lusid_drive_filename = 'instrument_subset.csv' # str | File name.
    x_lusid_drive_path = '/finbourne university/' # str | File path.
    content_length = len(subset_file.encode('UTF-8'))
    body = subset_file # str | 

    try:
        # [EARLY ACCESS] CreateFile: Uploads a file to Lusid Drive. If using an SDK, consider using the UploadAsStreamAsync function for larger files instead.
        api_response = api_instance.create_file(x_lusid_drive_filename, x_lusid_drive_path, content_length, body)
    except lusid_drive.rest.ApiException as e:
        if not exception_guard(e, 671):
            print("Exception when calling FilesApi->create_file: %s\n" % e)
        

# Luminesce Syntax

In this part of the course we will provide some additional technical information about luminesce, and demonstrate how luminesce queries can be used to extract information from your data.

## What is Structured Query Language (SQL)


SQL - pronounced "sequel" was first developed at IBM in the early 1970s, as a way to query and manipulate structured relational databases. SQL is a set-based, declarative programming language. Many implementations of SQL add extensions with procedural programming functionality, such as control-of-flow contructs.

## Luminesce Queries - SQL with extensions


SQLite is the most widely deployed database engine, used by several of the top web browsers, operating systems and application software.

Luminesce supports most of the query syntax for the SQLite dialect of SQL, but not the data manipulation or data definition syntax. Data can still be manipulated using some extensions to the language that Luminesce adds. 

Luminesce adds extensions to the SQLite implementation which can:
- exclude columns from queries
- automatically select important columns in queries
- parameterise queries
- write data to some Luminesce providers
- wait on results of some dependent operation, or for a time period
- run queries in parallel on an iterated range of parameters.



## Luminesce providers


A provider is a component that enables you to write a Luminesce SQL query for a data source in situ, without first having to extract, transform or load data from that source.

We supply providers for numerous data sources, including for the investment data stored in LUSID itself, for files stored in Drive and for some external data providers. You can also create your own provider for a proprietary data source.

There are two types of provider: a data provider and a direct provider.

A data provider is designed to query a data source whose shape is known. It therefore returns a table of results with a fixed number of fields (columns).

A direct provider is designed to query a data source whose shape is not known, and thus cannot return a table of results with a fixed number of fields (columns).

We'll use both data and direct providers in this tutorial.

## Use case: Using Luminesce to query and join data from disparate sources, and to distribute the resulting information.


I'd like to read a file from Drive and use the data from this file to filter transaction results. I'd like to join these filtered results with another provider and distribute the results. This will be packaged up into a simple to use View which runs on a schedule.

## An intro to SQLite queries

### A query using a simple SELECT statement


Its simple to get data from a provider using a select statement.

Use the following statement to get all data from a provider:

`SELECT * FROM SOME.PROVIDER`

Or retrieve data from a subset of columns by specifying them:

`SELECT A_COLUMN, ANOTHER_COLUMN FROM SOME.PROVIDER`

Here we'll use a SELECT statement to query for all of our porfolios.

In [8]:
%%luminesce
SELECT * 
FROM Lusid.Instrument.Equity
LIMIT 100

,LusidInstrumentId,ClientInternal,Figi,QuotePermId,EdiKey,Isin,Sedol,Cusip,Ticker,CompositeFigi,...,Scope,DisplayName,LookThroughPortfolioScope,LookThroughPortfolioCode,Type,InferredAssetClass,InferredDomCcy,IsActive,Identifiers,DomCcy
0,LUID_00003DAV,NaN,BBG000BVV7G1,NaN,NaN,US8825081040,NaN,882508104,NaN,NaN,...,fbn-uni-T02002,Texas Instruments,NaN,NaN,Equity,Equities,USD,True,NaN,USD
1,LUID_00003DM9,NaN,BBG000BDYNG3,NaN,NaN,GB00B7T77214,B7T7721,NaN,NaN,NaN,...,FBNUniversityModule-T2-2,NatWest Group,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
2,LUID_00003DEH,NaN,BBG000D2RM57,NaN,NaN,US1729081059,NaN,172908105,NaN,NaN,...,FBNUniversity,Cintas Corporation,NaN,NaN,Equity,Equities,USD,True,NaN,USD
3,LUID_00003DMK,NaN,BBG000C04D57,NaN,NaN,GB0031348658,3134865,NaN,NaN,NaN,...,FBNUniversityModule-T2-2,Barclays,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
4,LUID_00003DNB,NaN,BBG000FVFDY2,NaN,NaN,US58733R1023,NaN,58733R102,NaN,NaN,...,FBNUniversityModule-T2-2,MercadoLibre,NaN,NaN,Equity,Equities,USD,True,NaN,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LUID_00003DJV,NaN,BBG005CPNTQ2,NaN,NaN,US5007541064,NaN,500754106,NaN,NaN,...,FBNUniversityModule-T2-2,Kraft Heinz Co,NaN,NaN,Equity,Equities,USD,True,NaN,USD
96,LUID_00003DAD,NaN,BBG000C734W3,NaN,NaN,US75886F1075,NaN,75886F107,NaN,NaN,...,fbn-uni-T02002,Regeneron Pharmaceuticals,NaN,NaN,Equity,Equities,USD,True,NaN,USD
97,LUID_00003DCZ,NaN,BBG000BS1Q24,NaN,NaN,CH0198251305,B9895B7,NaN,NaN,NaN,...,FBNUniversity,Coca-Cola HBC,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
98,LUID_00003DG2,NaN,BBG000K6MRN4,NaN,NaN,USN070592100,NaN,NaN,NaN,NaN,...,FBNUniversity,ASML Holding,NaN,NaN,Equity,Equities,USD,True,NaN,USD


In this example, we've selected all columns and rows from the Lusid.Instrument.Equity provider. We can specify column names if we are only interested in some parts of the data.

In [9]:
%%luminesce
SELECT DisplayName, Figi 
FROM Lusid.Instrument.Equity
LIMIT 100

,DisplayName,Figi
0,KLA Corporation,BBG000BMTFR4
1,"Workday, Inc.",BBG000VC0T95
2,Cadence Design Systems,BBG000C13CD9
3,Apple Inc.,BBG000B9XRY4
4,RELX,BBG000C0KDR5
...,...,...
95,Intel Corporation,BBG000C0G1D1
96,eBay Inc.,BBG000BPJ466
97,Marriott International,BBG000BGD7W6
98,Informa,BBG000BLHQP7


Here I really just want to see the name of the Equity and the Figi, I've selected only those columns from the provider.

### Limiting the number of return results using LIMIT


These simple SELECT statements are useful for getting an idea of the shape and quality of the data returned by a provider, but they tend to return a lot of data. Lets use LIMIT to reduce the amount of data requested:

In [10]:
%%luminesce
SELECT DisplayName, Figi 
FROM Lusid.Instrument.Equity 
LIMIT 5

,DisplayName,Figi
0,Texas Instruments,BBG000BVV7G1
1,NatWest Group,BBG000BDYNG3
2,Cintas Corporation,BBG000D2RM57
3,Barclays,BBG000C04D57
4,MercadoLibre,BBG000FVFDY2


This example returns the first 5 rows of data from the Lusid.Instrument.Equity provider.

### Filter queries WHERE ...


We can now filter these results down - lets get all of the Equities that are listed in GBP


In [11]:
%%luminesce
SELECT * 
FROM Lusid.Instrument.Equity 
WHERE DomCcy = 'GBP'
LIMIT 100

,LusidInstrumentId,ClientInternal,Figi,QuotePermId,EdiKey,Isin,Sedol,Cusip,Ticker,CompositeFigi,...,Scope,DisplayName,LookThroughPortfolioScope,LookThroughPortfolioCode,Type,InferredAssetClass,InferredDomCcy,IsActive,Identifiers,DomCcy
0,LUID_00003DGB,NaN,BBG000C0KDR5,NaN,NaN,GB00B2B0DG97,B2B0DG9,NaN,NaN,NaN,...,FBNUniversity,RELX,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
1,LUID_00003DBM,NaN,BBG000C05R82,NaN,NaN,GB0030913577,3091357,NaN,NaN,NaN,...,fbn-uni-T02002,BT Group,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
2,LUID_00003DAX,NaN,BBG000C04D57,NaN,NaN,GB0031348658,3134865,NaN,NaN,NaN,...,fbn-uni-T02002,Barclays,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
3,LUID_00003D6U,NaN,BBG0027J63T2,NaN,NaN,GB00B71N6K86,B71N6K8,NaN,NaN,NaN,...,default,EVRAZ PLC,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
4,LUID_00003D7W,NaN,BBG000GVNZ28,NaN,NaN,GB0006043169,604316,NaN,NaN,NaN,...,fbn-uni-T02002,Morrisons,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LUID_00003D6S,NaN,BBG000BS69D5,NaN,NaN,GB0002374006,237400,NaN,NaN,NaN,...,default,DIAGEO PLC,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
96,LUID_00003DBV,NaN,BBG000H90Q58,NaN,NaN,GB00B18V8630,B18V863,NaN,NaN,NaN,...,fbn-uni-T02002,Pennon Group,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
97,LUID_00003DDO,NaN,BBG000C21Y87,NaN,NaN,GB00BBG9VN75,BBG9VN7,NaN,NaN,NaN,...,FBNUniversity,Aveva,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
98,LUID_00003DJU,NaN,BBG000BDXZH6,NaN,NaN,GB00B3MBS747,B3MBS74,NaN,NaN,NaN,...,FBNUniversityModule-T2-2,Ocado Group,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP


We've filtered our query for equities who have a domestic currency of Pound Sterling.

### JOINing disparate datasources with join


Now, I've got a file in drive with sector information for some of these instruments. I'm going to join data from the file using our Drive provider with the saved instrument data so I can see the sectors for instruments where we know this information.

First lets get our instrument data.

Then we'll grab the file contents from drive.

We'll take a look at cross-joins, which SQLite uses to build left joins and inner joins. Cross-joins return the cartesian product of both tables - each row on the left table is concatenated with every row on the right table, to form a massive table with every combination of both tables' rows:


In [12]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector 
FROM Lusid.Instrument.Equity as Equities 
CROSS JOIN @x as File
LIMIT 100

,DisplayName,DomCcy,Sector
0,Texas Instruments,USD,Financial Services
1,Texas Instruments,USD,General Industrials
2,Texas Instruments,USD,Travel & Leisure
3,Texas Instruments,USD,Equity Investment Instruments
4,Texas Instruments,USD,Household Goods & Home Construction
...,...,...,...
95,Barclays,GBP,Mining
96,Barclays,GBP,Media
97,Barclays,GBP,Mobile Telecommunications
98,Barclays,GBP,Food & Drug Retailers


We've returned a table with every combination of Lusid Equities with every row in our file.

In a left join, all rows from the left table are returned, along with rows in the right table that match the supplied key constraint. If there is a row in the left table with no matching row on the right, the columns that come from the right table in the resulting row are set to null.

In [13]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector, Equities.Figi as Figi 
FROM @x as File
LEFT OUTER JOIN Lusid.Instrument.Equity as Equities
on Equities.Figi = File.Figi
LIMIT 100

,DisplayName,DomCcy,Sector,Figi
0,Schroders,GBP,Financial Services,BBG000BF0TF3
1,Schroders,GBP,Financial Services,BBG000BF0TF3
2,Schroders,GBP,Financial Services,BBG000BF0TF3
3,DS Smith,GBP,General Industrials,BBG000BF1LF9
4,DS Smith,GBP,General Industrials,BBG000BF1LF9
...,...,...,...,...
90,Ocado Group,GBP,Food & Drug Retailers,BBG000BDXZH6
91,BRITISH LAND CO PLC,GBP,Real Estate Investment Trusts,BBG000BD7DW7
92,British Land,GBP,Real Estate Investment Trusts,BBG000BD7DW7
93,British Land,GBP,Real Estate Investment Trusts,BBG000BD7DW7


We see this here as there are some companies with no matching sector in our file - these are returned as null.

We are going to ignore instruments without a matching sector for now, so we'll use an inner join.
Inner joins only return rows where the joining key has a match on both sides of the table:

In [14]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector, Equities.Figi as Figi 
FROM @x as File 
INNER JOIN Lusid.Instrument.Equity as Equities  on File.Figi = Equities.Figi 
LIMIT 100


,DisplayName,DomCcy,Sector,Figi
0,Schroders,GBP,Financial Services,BBG000BF0TF3
1,Schroders,GBP,Financial Services,BBG000BF0TF3
2,Schroders,GBP,Financial Services,BBG000BF0TF3
3,DS Smith,GBP,General Industrials,BBG000BF1LF9
4,DS Smith,GBP,General Industrials,BBG000BF1LF9
...,...,...,...,...
90,Ocado Group,GBP,Food & Drug Retailers,BBG000BDXZH6
91,BRITISH LAND CO PLC,GBP,Real Estate Investment Trusts,BBG000BD7DW7
92,British Land,GBP,Real Estate Investment Trusts,BBG000BD7DW7
93,British Land,GBP,Real Estate Investment Trusts,BBG000BD7DW7


Now we have a joined table where only Equities with a sector on file are returned, along with their sector.

### GROUPing results BY some condition in query


We can also group our results by a column - we'll perform an aggregation and group-by to count the number of instrument in each sector.

There are many different aggregation functions in SQLite, such as sum, avg and count.

Sum will return the sum of the specified field in each group.
Average returns the mean of the specified field in each group.
Count returns the number of results that are non-null for the field in each group. We can also specify that count returns the count of distinct values [Count(Distinct <field>)], or all rows [count(*)], in each group.

In [15]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

SELECT count(File.Sector), File.Sector
FROM @x as File
LEFT OUTER JOIN Lusid.Instrument.Equity as Equities
on Equities.Figi = File.Figi
group by File.Sector
LIMIT 100

,count(File_Sector COLLATE BINARY),Sector
0,3,Automobiles & Parts
1,3,Equity Investment Instruments
2,6,Financial Services
3,3,Food & Drug Retailers
4,3,General Industrials
5,4,Household Goods & Home Construction
6,3,Investment Services
7,13,Life Insurance
8,3,Media
9,9,Mining


We've now grouped our results by sector, and for each sector, we''ve returned how many Equities belong to the sector.

## Introducing some Luminesce extensions


### Using ^ to SELECT the most important columns from a provider


Let's use the carat symbol to grab then more important columns in the Equity provider. The list of important columns is decided by the writer of the Provider.

In [16]:
%%luminesce
SELECT ^ 
FROM Lusid.Instrument.Equity
LIMIT 100

,LusidInstrumentId,ClientInternal,Figi,QuotePermId,EdiKey,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive
0,LUID_00003DAV,NaN,BBG000BVV7G1,NaN,NaN,fbn-uni-T02002,Texas Instruments,Equity,Equities,USD,True
1,LUID_00003DM9,NaN,BBG000BDYNG3,NaN,NaN,FBNUniversityModule-T2-2,NatWest Group,Equity,Equities,GBP,True
2,LUID_00003DEH,NaN,BBG000D2RM57,NaN,NaN,FBNUniversity,Cintas Corporation,Equity,Equities,USD,True
3,LUID_00003DMK,NaN,BBG000C04D57,NaN,NaN,FBNUniversityModule-T2-2,Barclays,Equity,Equities,GBP,True
4,LUID_00003DNB,NaN,BBG000FVFDY2,NaN,NaN,FBNUniversityModule-T2-2,MercadoLibre,Equity,Equities,USD,True
...,...,...,...,...,...,...,...,...,...,...,...
95,LUID_00003DJV,NaN,BBG005CPNTQ2,NaN,NaN,FBNUniversityModule-T2-2,Kraft Heinz Co,Equity,Equities,USD,True
96,LUID_00003DAD,NaN,BBG000C734W3,NaN,NaN,fbn-uni-T02002,Regeneron Pharmaceuticals,Equity,Equities,USD,True
97,LUID_00003DCZ,NaN,BBG000BS1Q24,NaN,NaN,FBNUniversity,Coca-Cola HBC,Equity,Equities,GBP,True
98,LUID_00003DG2,NaN,BBG000K6MRN4,NaN,NaN,FBNUniversity,ASML Holding,Equity,Equities,USD,True


In the Lusid Equities provider: LusidInstrumentId, Scope, DisplayName, InferredAssetClass, InferredDomCcy and IsActive are the fields marked by the provider as being most important.

### Excluding columns from the results of a query using the except keyword.


Using some providers, we can also exclude columns from the result of a query by using the except keyword:

In [17]:
%%luminesce
SELECT ^ (except Type, IsActive) 
FROM Lusid.Instrument.Equity
LIMIT 100

,LusidInstrumentId,ClientInternal,Figi,QuotePermId,EdiKey,Scope,DisplayName,InferredAssetClass,InferredDomCcy
0,LUID_00003DKZ,NaN,BBG000BMTFR4,NaN,NaN,FBNUniversityModule-T2-2,KLA Corporation,Equities,USD
1,LUID_00003DDS,NaN,BBG000VC0T95,NaN,NaN,FBNUniversity,"Workday, Inc.",Equities,USD
2,LUID_00003DFQ,NaN,BBG000C13CD9,NaN,NaN,FBNUniversity,Cadence Design Systems,Equities,USD
3,LUID_00003D7Y,NaN,BBG000B9XRY4,NaN,NaN,fbn-uni-T02002,Apple Inc.,Equities,USD
4,LUID_00003DGB,NaN,BBG000C0KDR5,NaN,NaN,FBNUniversity,RELX,Equities,GBP
...,...,...,...,...,...,...,...,...,...
95,LUID_00003DMI,NaN,BBG000C0G1D1,NaN,NaN,FBNUniversityModule-T2-2,Intel Corporation,Equities,USD
96,LUID_00003DJP,NaN,BBG000BPJ466,NaN,NaN,FBNUniversityModule-T2-2,eBay Inc.,Equities,USD
97,LUID_00003DM1,NaN,BBG000BGD7W6,NaN,NaN,FBNUniversityModule-T2-2,Marriott International,Equities,USD
98,LUID_00003DBU,NaN,BBG000BLHQP7,NaN,NaN,fbn-uni-T02002,Informa,Equities,GBP


Here we select the most important columns again, but exclude the Type and IsActive columns

### Using parameters to filter queries


Many providers will have parameters that can also be used to filter a Luminesce query as part of a standard WHERE clause.

We can see these parameters by querying the Sys.Field Provider.

In [18]:
%%luminesce
SELECT FieldName, DataType, ParamDefaultValue, Description 
FROM Sys.Field 
WHERE TableName = 'Lusid.Instrument.Equity' 
AND FieldType = 'Parameter'
LIMIT 100

,FieldName,DataType,ParamDefaultValue,Description
0,AsAt,DateTime,0001-01-01T00:00:00Z,As at time to query at. Defaults to latest.
1,EffectiveAt,DateTime,0001-01-01T00:00:00Z,Effective time to query at. Defaults to latest.
2,UseLusidFilter,Boolean,True,Should the filter be translated to a Finbourne...


We can see there are 3 fields we can filter our query on: AsAt, EffectiveAt and UseLusidFilter

In [19]:
%%luminesce

SELECT * 
FROM Lusid.Instrument.Equity
WHERE UseLusidFilter = False
LIMIT 100

,LusidInstrumentId,ClientInternal,Figi,QuotePermId,EdiKey,Isin,Sedol,Cusip,Ticker,CompositeFigi,...,Scope,DisplayName,LookThroughPortfolioScope,LookThroughPortfolioCode,Type,InferredAssetClass,InferredDomCcy,IsActive,Identifiers,DomCcy
0,LUID_00003DAV,NaN,BBG000BVV7G1,NaN,NaN,US8825081040,NaN,882508104,NaN,NaN,...,fbn-uni-T02002,Texas Instruments,NaN,NaN,Equity,Equities,USD,True,NaN,USD
1,LUID_00003DM9,NaN,BBG000BDYNG3,NaN,NaN,GB00B7T77214,B7T7721,NaN,NaN,NaN,...,FBNUniversityModule-T2-2,NatWest Group,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
2,LUID_00003DEH,NaN,BBG000D2RM57,NaN,NaN,US1729081059,NaN,172908105,NaN,NaN,...,FBNUniversity,Cintas Corporation,NaN,NaN,Equity,Equities,USD,True,NaN,USD
3,LUID_00003DMK,NaN,BBG000C04D57,NaN,NaN,GB0031348658,3134865,NaN,NaN,NaN,...,FBNUniversityModule-T2-2,Barclays,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
4,LUID_00003DNB,NaN,BBG000FVFDY2,NaN,NaN,US58733R1023,NaN,58733R102,NaN,NaN,...,FBNUniversityModule-T2-2,MercadoLibre,NaN,NaN,Equity,Equities,USD,True,NaN,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LUID_00003DJV,NaN,BBG005CPNTQ2,NaN,NaN,US5007541064,NaN,500754106,NaN,NaN,...,FBNUniversityModule-T2-2,Kraft Heinz Co,NaN,NaN,Equity,Equities,USD,True,NaN,USD
96,LUID_00003DAD,NaN,BBG000C734W3,NaN,NaN,US75886F1075,NaN,75886F107,NaN,NaN,...,fbn-uni-T02002,Regeneron Pharmaceuticals,NaN,NaN,Equity,Equities,USD,True,NaN,USD
97,LUID_00003DCZ,NaN,BBG000BS1Q24,NaN,NaN,CH0198251305,B9895B7,NaN,NaN,NaN,...,FBNUniversity,Coca-Cola HBC,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
98,LUID_00003DG2,NaN,BBG000K6MRN4,NaN,NaN,USN070592100,NaN,NaN,NaN,NaN,...,FBNUniversity,ASML Holding,NaN,NaN,Equity,Equities,USD,True,NaN,USD


Here we've filtered an Equity query on the UseLusidFilter. Notice there is no returned Field called UseLusidFilter - its not a column in our data.

## Using Luminesce variables


You can use variables to create and populate arbitrary tables of data that can then be used as part of a Luminesce query for either a data provider or a direct provider.

Declare a variable that represents a table of data using:
`@variable`

Or to represent a single value:
`@@variable`

Let's see variables in action. 

In [20]:
%%luminesce
@instruments_in_GBP = SELECT * 
                      FROM Lusid.Instrument.Equity 
                      WHERE DomCcy = 'GBP'
                      LIMIT 100;
SELECT Isin 
FROM @instruments_in_GBP;

,Isin
0,GB00B2B0DG97
1,GB0030913577
2,GB0031348658
3,GB00B71N6K86
4,GB0006043169
...,...
91,GB0002374006
92,GB00B18V8630
93,GB00BBG9VN75
94,GB00B3MBS747


Here_instruments_in GBP is a table variable, set from the results of the query on equities.

In [21]:
%%luminesce
@sectors = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

@@selected_sector = SELECT 'Life Insurance';
SELECT * 
FROM @sectors 
WHERE Sector = @@selected_sector
LIMIT 100

,Name,Sector,Figi
0,Prudential plc,Life Insurance,BBG000BDY322
1,St. James's Place plc,Life Insurance,BBG000BRMPC4
2,Legal & General,Life Insurance,BBG000BDQCG6
3,Aviva,Life Insurance,BBG000DDCSJ1


I've set selected_sector as a single-value variable "Lise Insurance", and filtered my file by sector.

## Introducing more advanced Luminesce extensions


You can use the CROSS APPLY or OUTER APPLY statement to iterate over a set of parameters. This is useful to execute a Luminesce query in parallel for each of the parameter inputs.

### Cross-apply

Cross-apply is similar to INNER JOIN, in that only records that have matching values on both sides are returned. 
We'll run some equity queries in parallel using Tools.split to create a set of currencies to filter by- the table will contain the currency joined to all of the returned equity tables:

In [22]:
%%luminesce

select VALUE as val
from Tools.Split 
where Original = 'GBP USD CHF' 
AND Delimiters = ' ';


,val
0,GBP
1,USD
2,CHF


First, we use Tools.Split to turn a space separated string of currencies into into a table where each currency is an entry in the table.

In [23]:
%%luminesce

@currencies = select VALUE as val
              from Tools.Split 
              where Original = 'GBP USD CHF' 
              AND Delimiters = ' ';

SELECT c.val as selectedCurrency, results.*
FROM @currencies c
CROSS APPLY (
       SELECT equity.^ 
       FROM Lusid.Instrument.Equity equity
       WHERE equity.DomCcy = c.val
       LIMIT 100       
) results


,selectedCurrency,LusidInstrumentId,ClientInternal,Figi,QuotePermId,EdiKey,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive,__row_key__
0,GBP,LUID_00003DM9,NaN,BBG000BDYNG3,NaN,NaN,FBNUniversityModule-T2-2,NatWest Group,Equity,Equities,GBP,True,1
1,GBP,LUID_00003DMK,NaN,BBG000C04D57,NaN,NaN,FBNUniversityModule-T2-2,Barclays,Equity,Equities,GBP,True,1
2,GBP,LUID_00003D7H,NaN,BBG000BD7DW7,NaN,NaN,fbn-uni-T02002,British Land,Equity,Equities,GBP,True,1
3,GBP,LUID_00003D5Y,NaN,some-figi-code-2,NaN,NaN,fbn-uni-T02002,fbn-uni-test-example,Equity,Equities,GBP,True,1
4,GBP,LUID_00003DFH,NaN,BBG000C6K6G9,NaN,NaN,FBNUniversity,Vodafone Group,Equity,Equities,GBP,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,USD,LUID_00003DE9,NaN,BBG000BJKPG0,NaN,NaN,FBNUniversity,"Fiserv, Inc.",Equity,Equities,USD,True,2
91,USD,LUID_00003DMI,NaN,BBG000C0G1D1,NaN,NaN,FBNUniversityModule-T2-2,Intel Corporation,Equity,Equities,USD,True,2
92,USD,LUID_00003DJP,NaN,BBG000BPJ466,NaN,NaN,FBNUniversityModule-T2-2,eBay Inc.,Equity,Equities,USD,True,2
93,USD,LUID_00003DM1,NaN,BBG000BGD7W6,NaN,NaN,FBNUniversityModule-T2-2,Marriott International,Equity,Equities,USD,True,2


We then save the result of that query in a variable @currency, which is passed as a criterion in the CROSS APPLY subquery,  which runs in parallel for each row in @currency.

This example works similar to an inner join, we've run a query in parallel for some currency values, returning a similar table to an inner join. Remember, inner join returns only rows which fulfil the join criteria.

### Outer-apply


Outer apply is similar to LEFT OUTER JOIN, in that all records on the left side are returned, even when there are no matching records on the right side.

In [24]:
%%luminesce

@currencies = select VALUE as val
              from Tools.Split 
              where Original = 'GBP USD CHF' 
              AND Delimiters = ' ';
              
SELECT c.val, results.*
FROM @currencies c
OUTER APPLY (
       SELECT equity.^ 
       FROM Lusid.Instrument.Equity as equity
       WHERE equity.DomCcy = c.val
       LIMIT 100
) results


,val,LusidInstrumentId,ClientInternal,Figi,QuotePermId,EdiKey,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive,__row_key__
0,GBP,LUID_00003D57,imd_23536673,BBG000BN0PP3,NaN,NaN,FBNUniversityModule-1,Sage Group,Equity,Equities,GBP,True,1.0
1,GBP,LUID_00003D58,imd_34539955,BBG000FBCMD2,NaN,NaN,FBNUniversityModule-1,Experian,Equity,Equities,GBP,True,1.0
2,GBP,LUID_00003D5L,NaN,BBG001MM1KV4,NaN,NaN,FBNUniversityModule-3-1,Glencore,Equity,Equities,GBP,True,1.0
3,GBP,LUID_00003D5X,NaN,BBG000C05BD1,NaN,NaN,FBNUniversityModule-6-3,BP,Equity,Equities,GBP,True,1.0
4,GBP,LUID_00003D61,NaN,BBG000PG2GZ0,NaN,NaN,default,ADMIRAL GROUP PLC,Equity,Equities,GBP,True,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,USD,LUID_00003DM1,NaN,BBG000BGD7W6,NaN,NaN,FBNUniversityModule-T2-2,Marriott International,Equity,Equities,USD,True,2.0
102,USD,LUID_00003DMI,NaN,BBG000C0G1D1,NaN,NaN,FBNUniversityModule-T2-2,Intel Corporation,Equity,Equities,USD,True,2.0
103,USD,LUID_00003DNA,NaN,BBG000BBJQV0,NaN,NaN,FBNUniversityModule-T2-2,Nvidia Corporation,Equity,Equities,USD,True,2.0
104,USD,LUID_00003DNB,NaN,BBG000FVFDY2,NaN,NaN,FBNUniversityModule-T2-2,MercadoLibre,Equity,Equities,USD,True,2.0


We've run a query in parallel for some currency values, returning a similar table to a left join with our selected currencies on the left side of the join. Here, the values from Lusid.Instrument.Equity on the CHF row are null.